Описание задания:
В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с сайта. Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.

In [103]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, classification_report, confusion_matrix, recall_score, precision_score, f1_score


1. Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)

In [104]:
data = pd.read_csv('heart.csv')
data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


2. Подготовьте датасет к обучению моделей:
* Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.

In [105]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [106]:
data_encoded = pd.get_dummies(data, columns=['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'])
data_encoded.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_F,Sex_M,ChestPainType_ASY,...,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,1,0,0,...,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,0,1,0,...,0,0,0,0,1,1,0,0,0,1
3,48,138,214,0,108,1.5,1,1,0,1,...,0,0,0,1,0,0,1,0,1,0
4,54,150,195,0,122,0.0,0,0,1,0,...,1,0,0,1,0,1,0,0,0,1


In [107]:
data_encoded.columns

Index(['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak',
       'HeartDisease', 'Sex_F', 'Sex_M', 'ChestPainType_ASY',
       'ChestPainType_ATA', 'ChestPainType_NAP', 'ChestPainType_TA',
       'RestingECG_LVH', 'RestingECG_Normal', 'RestingECG_ST',
       'ExerciseAngina_N', 'ExerciseAngina_Y', 'ST_Slope_Down',
       'ST_Slope_Flat', 'ST_Slope_Up'],
      dtype='object')

3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

In [108]:
X = data_encoded.drop('HeartDisease', axis=1)
y = data_encoded['HeartDisease']

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

4. Обучите модель логистической регрессии с параметрами по умолчанию.

In [110]:
# Алгоритм логистической регрессии не отработал за max_iter=100 по умолчанию

model = LogisticRegression(max_iter=5000, random_state=42) 
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,5000
,multi_class,'deprecated'


In [111]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1])

5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:
cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’])

In [112]:
scoring_metrics = ['accuracy', 'recall', 'precision', 'f1']

In [113]:
cv_results = cross_validate(
    model,
    X=X,
    y=y,
    cv=10,
    scoring=scoring_metrics,
    n_jobs=-1,
    return_train_score=True
)

In [114]:
print(f'''Среднее по метрикам кросс-валидации LogisticRegression
    Test Accuracy: {cv_results['test_accuracy'].mean():.4f},
    Train Accuracy: {cv_results['train_accuracy'].mean():.4f},

    Test Recall: {cv_results['test_recall'].mean():.4f},
    Train Recall: {cv_results['train_recall'].mean():.4f},

    Test Precision: {cv_results['test_precision'].mean():.4f},
    Train Precision: {cv_results['train_precision'].mean():.4f},

    Test F1: {cv_results['test_f1'].mean():.4f},
    Train F1: {cv_results['train_f1'].mean():.4f}
''')

Среднее по метрикам кросс-валидации LogisticRegression
    Test Accuracy: 0.8527,
    Train Accuracy: 0.8717,

    Test Recall: 0.8715,
    Train Recall: 0.8996,

    Test Precision: 0.8690,
    Train Precision: 0.8725,

    Test F1: 0.8659,
    Train F1: 0.8858,



6. Оптимизируйте 3-4 параметра модели:
* Используйте GridSearchCV.
* Используйте RandomizedSearchCV.
* Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.
* Повторите п. 5 после каждого итогового изменения параметров.

In [115]:
# GridSearchCV для LogisticRegression

param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['saga']
}

grid_search = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Лучшие параметры GridSearchCV LogisticRegression:")
print(grid_search.best_params_)

best_model = grid_search.best_estimator_

scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'f1': make_scorer(f1_score)
}

cv_results = cross_validate(
    best_model,
    X,
    y,
    cv=10,
    scoring=scoring_metrics,
    n_jobs=-1,
    return_train_score=True
)

print(f'''Среднее по метрикам кросс-валидации LogisticRegression
    Test Accuracy: {cv_results['test_accuracy'].mean():.4f},
    Train Accuracy: {cv_results['train_accuracy'].mean():.4f},

    Test Recall: {cv_results['test_recall'].mean():.4f},
    Train Recall: {cv_results['train_recall'].mean():.4f},

    Test Precision: {cv_results['test_precision'].mean():.4f},
    Train Precision: {cv_results['train_precision'].mean():.4f},

    Test F1: {cv_results['test_f1'].mean():.4f},
    Train F1: {cv_results['train_f1'].mean():.4f}
''')

Лучшие параметры GridSearchCV LogisticRegression:
{'C': 1, 'penalty': 'l2', 'solver': 'saga'}
Лучшая точность: 0.8624
Среднее по метрикам кросс-валидации LogisticRegression
    Test Accuracy: 0.8462,
    Train Accuracy: 0.8625,

    Test Recall: 0.8617,
    Train Recall: 0.8878,

    Test Precision: 0.8640,
    Train Precision: 0.8669,

    Test F1: 0.8585,
    Train F1: 0.8772,



In [116]:
# RandomizedSearchCV для LogisticRegression

param_randomsearch = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['saga']
}

random_search = RandomizedSearchCV(
    model,
    param_distributions=param_randomsearch,
    n_iter=12,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42 
)

random_search.fit(X_train, y_train)

print("Лучшие параметры RandomizedSearchCV LogisticRegression:")
print(random_search.best_params_)

best_model = random_search.best_estimator_

scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'f1': make_scorer(f1_score)
}

cv_results = cross_validate(
    best_model,
    X,
    y,
    cv=10,
    scoring=scoring_metrics,
    n_jobs=-1,
    return_train_score=True
)

print(f'''Среднее по метрикам кросс-валидации LogisticRegression
    Test Accuracy: {cv_results['test_accuracy'].mean():.4f},
    Train Accuracy: {cv_results['train_accuracy'].mean():.4f},

    Test Recall: {cv_results['test_recall'].mean():.4f},
    Train Recall: {cv_results['train_recall'].mean():.4f},

    Test Precision: {cv_results['test_precision'].mean():.4f},
    Train Precision: {cv_results['train_precision'].mean():.4f},

    Test F1: {cv_results['test_f1'].mean():.4f},
    Train F1: {cv_results['train_f1'].mean():.4f}
''')

Лучшие параметры RandomizedSearchCV (LogisticRegression):
{'solver': 'saga', 'penalty': 'l2', 'C': 1}
Лучшая точность: 0.8624
Среднее по метрикам кросс-валидации LogisticRegression
    Test Accuracy: 0.8462,
    Train Accuracy: 0.8625,

    Test Recall: 0.8617,
    Train Recall: 0.8878,

    Test Precision: 0.8640,
    Train Precision: 0.8669,

    Test F1: 0.8585,
    Train F1: 0.8772,



7. Сформулируйте выводы по проделанной работе:
* Сравните метрики построенных моделей.
* Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».